In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import time
%matplotlib inline
import os, shutil
import datetime
from hbp_nrp_virtual_coach.virtual_coach import VirtualCoach
vc = VirtualCoach(environment='local')

INFO: [2018-03-01 16:27:14,658 - rospy.topics] topicmanager initialized


INFO: [2018-03-01 16:27:14,811 - Configuration] Loading configuration file config.json
INFO: [2018-03-01 16:27:14,814 - Configuration] Using user specified environment: local
INFO: [2018-03-01 16:27:14,871 - VirtualCoach] Ready.


## Helper Functions
Some helper functions to calculate the fitness function, plot the robot's trajectory and the wheel speeds. These functions are specific to this experiment.

In [2]:
def fitness_function(wheel_speeds, trajectory):
    left_wheel = ([float(t[1]) for t in wheel_speeds[11:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[11:]])
    fitness = 0
    for i in range(len(left_wheel)):
        fitness += (left_wheel[i] + right_wheel[i])
    for xy in trajectory[11:]:
        if float(xy[0])>3.5 or float(xy[0])<-3.5 or float(xy[1])>2.5 or float(xy[1])<-2.5:
            fitness = -1
            print('Candidate out of bounds at '+str(xy))
    print(np.maximum(0, fitness/float(2*len(left_wheel))))
    return np.maximum(0, fitness/float(2*len(left_wheel)))

def get_top_performers(population, fitness_log, num_performers=8):
    """
    Extract the indices of the top individuals from the fitness log.
    
    :param fitness_log: fitness function scores for all individuals in a population
    :param num_performers: number for top performers to look for. Default value is
                           15, which corresponds to a truncation threshold of 25% in
                           this experiment.
    """
    top_performers = []
    for i in range(num_performers):
        max_index = np.argmax(fitness_log)
        print max_index
        top_performers.append(population[max_index])
        fitness_log[max_index] = -1

    return top_performers

def plot_trajectory(trajectory):
    plt.figure()
    plt.gca()
    plt.xticks([], [])
    plt.yticks([], [])
    plt.ylim(-3, 3)
    plt.xlim(-3.9, 3.9)
    x_axis = [x[0] for x in trajectory[2:]]
    y_axis = [y[1] for y in trajectory[2:]]
    plt.plot([float(x) for x in x_axis], [float(y) for y in y_axis])

def plot_wheel_speeds(wheel_speeds):
    left_wheel = ([float(t[1]) for t in wheel_speeds[11:]])
    right_wheel = ([float(t[2]) for t in wheel_speeds[11:]])
    plt.plot(range(len(left_wheel)), left_wheel, 'b')
    plt.plot(range(len(right_wheel)), right_wheel, 'r')

Evolutionary Algorithm helper functions

In [3]:
def one_point_crossover(parent1, parent2):
        parent1 = parent1.reshape(290)
        parent2 = parent2.reshape(290)
        child1 = np.zeros(290, dtype=int)
        child2 = np.zeros(290, dtype=int)
        point = np.random.randint(len(parent1))
        for i in range(point):
            child1[i] = parent1[i]
            child2[i] = parent2[i]
        for i in range(point, 290):
            child1[i] = parent2[i]
            child2[i] = parent1[i]
        child1 = child1.reshape(10, 29)
        child2 = child2.reshape(10, 29)
        return child1, child2
    
def bit_mutation(population):
        for individual in population:
            individual = individual.reshape(290)
            for i in range(290):
                if np.random.rand() < 0.05:
                    individual[i] = 0 if individual[i] else 1
        return population
    
def get_unique_pairs(population):
    pairs = []
    for i in range(len(population)):
        for j in range(i+1, len(population)):
            pairs.append((i, j))
    return pairs
            
def evolve_new_generation(top_performers):
    population = []
    for i in range(len(top_performers)):
        population.append(top_performers[i])
            
    pairs = get_unique_pairs(population)
    for i in pairs:
        parent1 = population[i[0]]
        parent2 = population[i[1]]
        child1, child2 = one_point_crossover(parent1, parent2)
        population.append(child1)
        population.append(child2)

    population = bit_mutation(population)
    rand = np.random.randint(len(population)-8)+8
    population[rand] = top_performers[0]
    return population

## The Brain
The PyNN script that creates the neural network stored as a string. A new binary genetic string that encodes the connections between neurons is passed on each run.

In [4]:
brain = """from hbp_nrp_cle.brainsim import simulator as sim
import numpy as np
import logging

logger = logging.getLogger(__name__)

dna = np.array([int(x) for x in '%s'.split(',')]).reshape(10, 29)

receptors = []
for r in range(1,19):
    receptors.append(np.nonzero(dna[:,r])[0])


def create_brain():

    NEURONPARAMS = {'v_rest': -60.5,
                    'tau_m': 4.0,
                    'tau_refrac': 2.0,
                    'tau_syn_E': 10.0,
                    'tau_syn_I': 10.0,
                    'e_rev_E': 0.0,
                    'e_rev_I': -75.0,
                    'v_thresh': -60.4,
                    'v_reset': -60.5}

    SYNAPSE_PARAMS = {"weight": 1.0,
                      "delay": 2.0}

    population = sim.Population(10, sim.IF_cond_alpha())
    population[0:10].set(**NEURONPARAMS)


    # Connect neurons
    CIRCUIT = population

    SYN = sim.StaticSynapse(**SYNAPSE_PARAMS)

    row_counter=0
    for row in dna:
    	logger.info(row)
        n = np.array(row)
        r_type = 'excitatory'
        if n[0]==0:
            r_type = 'inhibitory'
        for i in range(19,29):
            if n[i]==1:
                sim.Projection(presynaptic_population=CIRCUIT[row_counter:1+row_counter], postsynaptic_population=CIRCUIT[i-19:i-18], connector=sim.OneToOneConnector(), synapse_type=SYN, receptor_type=r_type)
        
        row_counter+=1

    sim.initialize(population, v=population.get('v_rest'))

    logger.debug("Circuit description: " + str(population.describe()))

    return population


circuit = create_brain()

"""

In [5]:
display_trial_tf = """@nrp.Robot2Neuron()
def display_trial_number(t):
    clientLogger.advertise('%s')
"""

## Run Experiment

In [6]:
class FloreanoExperiment(object):
    
    def __init__(self, population, generations):
        self.last_status = [None]
        self.population = population
        self.fitness_log = []
        self.sim = None
        self.started = False
        self.generations = generations
        self.sim_data = []

    def wait_condition(self, timeout, condition):
        start = time.time()
        while time.time() < start + timeout:
            time.sleep(0.25)
            if condition(self.last_status[0]):
                return
        raise Exception('Condition check failed')
        
    def on_status(self, status):
        self.last_status[0] = status
                 
    def save_simulation_data(self, trial, j):
        self.sim_data[trial].append([])
        wheel_speeds = self.sim.get_csv_data('wheel_speeds.csv')
        try:
            left_wheel = ([float(t[1]) for t in wheel_speeds[11:]])
        except Exception as inst:
            print(type(inst))
            print(inst.args)
            print(inst)
            for t in wheel_speeds:
                print(t)
        right_wheel = ([float(t[2]) for t in wheel_speeds[11:]])
        trajectory = self.sim.get_csv_data('robot_position.csv')
        fitness = fitness_function(wheel_speeds, trajectory)
        self.sim_data[trial][-1] = {
            'fitness': fitness,
            'wheel_speeds': wheel_speeds,
            'left_wheel': left_wheel,
            'right_wheel': right_wheel,
            'trajectory': trajectory
        }

        fd = open('SimData/full_dump.txt', 'a')
        fd.write('\n----------------------\n')
        fd.write(str(trial) + '-' + str(j))
        fd.write('\n----------------------\n')
        fd.write(str(self.sim_data[trial][-1]))
        fd.close

    def run_experiment(self):
        try:
            src_dir= '/home/hedgehog/Documents/Floreano/SimData/'
            dst_dir= os.path.join('/home/hedgehog/Documents/Floreano/SimData/', str(datetime.datetime.now()))
            shutil.copytree(src_dir, dst_dir)
            open('/home/hedgehog/Documents/Floreano/SimData/full_dump.txt', 'w').close()
            open('/home/hedgehog/Documents/Floreano/SimData/populations.txt', 'w').close()
            open('/home/hedgehog/Documents/Floreano/SimData/top_performers.txt', 'w').close()
        except:
            print('Error')
        
        fp = open('SimData/populations.txt', 'a')        
        for i in range(len(self.population)):
            np.savetxt(fp, self.population[i], header='0-'+str(i), fmt = '%d')
        fp.close()
        
        try:
            self.sim = vc.launch_experiment('floreano')
        except:
            time.sleep(1)
        self.sim.register_status_callback(self.on_status)
        for i in range(self.generations):
            self.sim_data.append([])
            for j in range(36):
                print("Generation {}, Population {}".format(i, j))
                genetic_string = ','.join(str(x) for x in population[j].ravel())
                self.sim.edit_brain(brain % genetic_string)
                self.sim.add_transfer_function(display_trial_tf % "Generation {}, Population {}".format(i, j) )
                self.sim.start()
                # run simulation for 40 seconds
                self.wait_condition(1000, lambda x: x['simulationTime'] > 40)
                self.sim.pause()
                self.save_simulation_data(i,j)
                self.sim.reset('full')
                self.wait_condition(100, lambda x: x['state'] == 'paused' and x['simulationTime'] == 0)
            self.fitness_log = [result['fitness'] for result in floreano_experiment.sim_data[i]]
            self.top_performers = get_top_performers(self.population, list(self.fitness_log))
            ft = open('SimData/top_performers.txt', 'a')
            for j in range(len(self.top_performers)):
                np.savetxt(ft, self.top_performers[j], header=str(i)+'-'+str(j), fmt = '%d')
            ft.close()
            self.population = evolve_new_generation(self.top_performers)
            fp = open('SimData/populations.txt', 'a')
            for j in range(len(self.population)):
                np.savetxt(fp, self.population[j], header=str(i+1)+'-'+str(j), fmt = '%d')
            fp.close()

In [ ]:
population = np.random.randint(2, size=(36, 10, 29)) # random population of 10 binary genetic strings
floreano_experiment = FloreanoExperiment(population, 30)
floreano_experiment.run_experiment()

INFO: [2018-03-01 16:27:20,598 - VirtualCoach] Preparing to launch floreano.
INFO: [2018-03-01 16:27:20,599 - VirtualCoach] Retrieving list of experiments.
INFO: [2018-03-01 16:27:20,619 - VirtualCoach] Retrieving list of available servers.
[{u'gzweb': {u'assets': u'http://localhost:8080/assets',
             u'nrp-services': u'http://localhost:8080',
             u'videoStreaming': u'http://localhost:8080/webstream/',
             u'websocket': u'ws://localhost:8080/gzbridge'},
  u'id': u'localhost',
  u'rosbridge': {u'websocket': u'ws://localhost:8080/rosbridge'},
  u'serverJobLocation': u'local'}]
INFO: [2018-03-01 16:27:20,627 - Simulation] Attempting to launch floreano on localhost.
INFO: [2018-03-01 16:27:28,832 - Simulation (floreano - localhost #0)] Simulation Successfully Created.
INFO: [2018-03-01 16:27:28,867 - Simulation (floreano - localhost #0)] Ready.
INFO: [2018-03-01 16:27:28,868 - Simulation (floreano - localhost #0)] Status callback registered.
Generation 0, Populati

INFO: [2018-03-01 16:29:05,213 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2018-03-01 16:29:05,221 - Simulation (floreano - localhost #0)] Transfer Function '' successfully updated
INFO: [2018-03-01 16:29:05,221 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2018-03-01 16:29:05,225 - Simulation (floreano - localhost #0)] Simulation state: started
INFO: [2018-03-01 16:29:48,768 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-03-01 16:29:48,772 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-03-01 16:29:48,774 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 16:29:48,794 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 16:29:48,811 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01

INFO: [2018-03-01 16:31:29,272 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2018-03-01 16:31:29,472 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2018-03-01 16:31:29,693 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2018-03-01 16:31:30,224 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
Generation 0, Population 5
INFO: [2018-03-01 16:31:31,226 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 16:31:31,231 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2018-03-01 16:31:31,232 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 16:31:31,237 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 16:31:31,241 - Simulation (floreano - localhost #0)]

INFO: [2018-03-01 16:33:56,930 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-03-01 16:33:56,935 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-03-01 16:33:56,936 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 16:33:56,990 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 16:33:57,013 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 16:33:57,048 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
1.0555725608e-05
INFO: [2018-03-01 16:33:57,072 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-03-01 16:33:57,077 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-03-01 16:33:57,079 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2018-0

INFO: [2018-03-01 16:34:47,339 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-03-01 16:34:47,341 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2018-03-01 16:34:48,546 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2018-03-01 16:34:48,547 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the 3D world
INFO: [2018-03-01 16:34:48,551 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2018-03-01 16:34:48,855 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2018-03-01 16:34:49,225 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2018-03-01 16:34:49,425 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2018-03-01 16:34:49,652 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
IN

...

INFO: [2018-03-01 19:04:33,497 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-03-01 19:04:33,498 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2018-03-01 19:04:34,614 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2018-03-01 19:04:34,615 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the 3D world
INFO: [2018-03-01 19:04:34,622 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight_0
INFO: [2018-03-01 19:04:34,926 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: plane
INFO: [2018-03-01 19:04:35,324 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: pointlight_0
INFO: [2018-03-01 19:04:35,527 - Simulation (floreano - localhost #0)] [Resetting the simulation] Loading: plane
INFO: [2018-03-01 19:04:35,761 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
IN

INFO: [2018-03-01 19:05:32,764 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2018-03-01 19:05:32,768 - Simulation (floreano - localhost #0)] Simulation state: started
INFO: [2018-03-01 19:06:24,067 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-03-01 19:06:24,073 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-03-01 19:06:24,074 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 19:06:24,103 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 19:06:24,133 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 19:06:24,162 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
7.18984350068e-06
INFO: [2018-03-01 19:06:24,187 - Simulation (floreano - localhost #0)] Attempting to transition to state: 

INFO: [2018-03-01 19:08:16,052 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the brain
INFO: [2018-03-01 19:08:16,558 - Simulation (floreano - localhost #0)] Reset completed. The simulation has been paused and will not be startedautomatically.
Generation 5, Population 4
INFO: [2018-03-01 19:08:16,810 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:08:16,821 - Simulation (floreano - localhost #0)] Attempting to set Brain
INFO: [2018-03-01 19:08:16,822 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:08:16,829 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:08:16,834 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:08:16,839 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:08:17,275 - Simulation (floreano - localhost #0)] Brain successfully updated.
INFO

INFO: [2018-03-01 19:11:00,728 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 19:11:00,752 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
INFO: [2018-03-01 19:11:00,776 - Simulation (floreano - localhost #0)] Attempting to retrieve recorders' csv data
0.260425830158
INFO: [2018-03-01 19:11:00,806 - Simulation (floreano - localhost #0)] Attempting to transition to state: paused
INFO: [2018-03-01 19:11:00,812 - Simulation (floreano - localhost #0)] Simulation state: paused
INFO: [2018-03-01 19:11:00,813 - Simulation (floreano - localhost #0)] Attempting to reset full
INFO: [2018-03-01 19:11:01,914 - Simulation (floreano - localhost #0)] [Resetting the simulation] 
INFO: [2018-03-01 19:11:01,917 - Simulation (floreano - localhost #0)] [Resetting the simulation] Restoring the 3D world
INFO: [2018-03-01 19:11:01,924 - Simulation (floreano - localhost #0)] [Resetting the simulation] Deleting: pointlight

INFO: [2018-03-01 19:12:56,899 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:12:56,904 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:12:56,910 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:12:56,918 - Simulation (floreano - localhost #0)] Attempting to retrieve brain
INFO: [2018-03-01 19:12:57,425 - Simulation (floreano - localhost #0)] Brain successfully updated.
INFO: [2018-03-01 19:12:57,426 - Simulation (floreano - localhost #0)] Attempting to retrieve transfer-function
INFO: [2018-03-01 19:12:57,442 - Simulation (floreano - localhost #0)] Attempting to set Transfer Function 
INFO: [2018-03-01 19:12:57,450 - Simulation (floreano - localhost #0)] Transfer Function '' successfully updated
INFO: [2018-03-01 19:12:57,451 - Simulation (floreano - localhost #0)] Attempting to transition to state: started
INFO: [2018-03-01 19:12:57,455 - Simulation (floreano 

In [8]:
fig, axes = plt.subplots(len(floreano_experiment.sim_data), 2)
for i in range(len(floreano_experiment.sim_data)):
    axes[i, 0].set_ylim(-3, 3)
    axes[i, 0].set_xlim(-3.9, 3.9)
    axes[i, 0].set_xticks([], [])
    axes[i, 0].set_yticks([], [])
    x_axis = [x[0] for x in floreano_experiment.sim_data[i]['trajectory'][11:]]
    y_axis = [y[1] for y in floreano_experiment.sim_data[i]['trajectory'][11:]]
    axes[i, 0].plot([float(x) for x in x_axis], [float(y) for y in y_axis])

    left_wheel = ([float(t[1]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][11:]])
    right_wheel = ([float(t[2]) for t in floreano_experiment.sim_data[i]['wheel_speeds'][11:]])
    axes[i, 1].plot(range(len(left_wheel)), left_wheel, 'b', label='Left Wheel')
    axes[i, 1].plot(range(len(right_wheel)), right_wheel, 'r', label='Right Wheel')
    axes[i, 1].set_ylim(-5, 5)
    axes[i, 1].set_xlabel('Time [ms]')
    axes[i, 1].set_ylabel('Speed m/s')

axes[0, 0].set_title('Robot Trajectory')
axes[0, 1].set_title('Wheel Speeds')
fig.set_figheight(25)
fig.set_figwidth(10)


TypeError: expected a string or other character buffer object